# Interacting with model services

We've seen how to deploy machine learning pipelines into production with `s2i` and now we'll see how we can use these services to make predictions.

In [39]:
import requests
from urllib.parse import urlencode
import json

DEFAULT_BASE_URL = "http://localhost:8080/%s"

def score_text(text, url = None):
    url = (url or (DEFAULT_BASE_URL % "predict")) 
    if type(text) == str:
        text = [text]
    payload = urlencode({"json_args" : json.dumps(text)})
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    response = requests.request("POST", url, data=payload, headers=headers)
    return json.loads(response.text)

def get_metrics(url = None):
    def parse_one_metric(line):
        ll = line.rsplit(' ', 1)
        return (ll[0], float(ll[1]))
    
    url = (url or (DEFAULT_BASE_URL % "metrics")) 
    response = requests.request("POST", url)
    return dict([parse_one_metric(line) for line in response.text.split('\n') if len(line) > 0 and line[0] != '#'])

In [40]:
score_text("It is a truth universally acknowledged")

['legitimate']

In [41]:
print(get_metrics())

{'python_gc_objects_collected_total{generation="0"}': 717.0, 'python_gc_objects_collected_total{generation="1"}': 35.0, 'python_gc_objects_collected_total{generation="2"}': 0.0, 'python_gc_objects_uncollectable_total{generation="0"}': 0.0, 'python_gc_objects_uncollectable_total{generation="1"}': 0.0, 'python_gc_objects_uncollectable_total{generation="2"}': 0.0, 'python_gc_collections_total{generation="0"}': 249.0, 'python_gc_collections_total{generation="1"}': 22.0, 'python_gc_collections_total{generation="2"}': 2.0, 'python_info{implementation="CPython",major="3",minor="6",patchlevel="6",version="3.6.6"}': 1.0, 'process_virtual_memory_bytes': 650973184.0, 'process_resident_memory_bytes': 104599552.0, 'process_start_time_seconds': 1556740178.16, 'process_cpu_seconds_total': 1.65, 'process_open_fds': 11.0, 'process_max_fds': 1048576.0, 'pipeline_processing_seconds_count': 20.0, 'pipeline_processing_seconds_sum': 0.3145384999952512, 'pipeline_processing_seconds_created': 1556740179.26194